#### ZJ_Zhang_w10_assn.py

### Task1: Create the csv file

In [1]:
import numpy as np
import pandas as pd
import re

# read the csv file
df = pd.read_csv('https://raw.githubusercontent.com/zhijing-zhang/DAV-5400/master/Arrival_delays.csv')
df

,Unnamed: 0,Unnamed: 1,Los Angeles,Phoenix,San Diego,San Francisco,Seattle
0,ALASKA,on time,497.0,221.0,212.0,503.0,1841.0
1,NaN,delayed,62.0,12.0,20.0,102.0,305.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AMWEST,on time,694.0,4840.0,383.0,320.0,201.0
4,NaN,delayed,117.0,415.0,65.0,129.0,61.0


### Task2 : Wide to long

Read the information from CSV file, and need to tidy and transform data

- delete the empty row
- change the name of the columns to identify every column
- **melt()** used to column to be as the grouping indicator
- **fillna()** used to fill the NAN as the preceding row
- **sort_values()** used to change the order of long by airline and state
- **reset_index()** used to reset the index

In [2]:
# delete the row with all NaN
df = df.dropna(axis=0,how='all')  
# add the column name for the unnamed column
df.rename( columns={'Unnamed: 0':'Airline','Unnamed: 1':'State'}, inplace=True )
df

C:\Users\lenovo\Anaconda3\lib\site-packages\pandas\core\frame.py:4223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


,Airline,State,Los Angeles,Phoenix,San Diego,San Francisco,Seattle
0,ALASKA,on time,497.0,221.0,212.0,503.0,1841.0
1,NaN,delayed,62.0,12.0,20.0,102.0,305.0
3,AMWEST,on time,694.0,4840.0,383.0,320.0,201.0
4,NaN,delayed,117.0,415.0,65.0,129.0,61.0


In [3]:
# Convert the table to Long format using Month & Category as the keys 
# and Airline, State, and name of destinations as the data values
long = df.melt(id_vars = ['Airline', 'State'], value_vars = ['Los Angeles','Phoenix','San Diego','San Francisco','Seattle'])

# Based on the content of the original data frame, 
# it appears that the 'NaN' values in the 'Airline' column should be filled with the 'Airline' value from the preceding row
long['Airline'].fillna(method='ffill', inplace = True)

# rename
long.rename( columns={'variable':'Destination','value':'Flights'}, inplace=True )
long

,Airline,State,Destination,Flights
0,ALASKA,on time,Los Angeles,497.0
1,ALASKA,delayed,Los Angeles,62.0
2,AMWEST,on time,Los Angeles,694.0
3,AMWEST,delayed,Los Angeles,117.0
4,ALASKA,on time,Phoenix,221.0
5,ALASKA,delayed,Phoenix,12.0
6,AMWEST,on time,Phoenix,4840.0
7,AMWEST,delayed,Phoenix,415.0
8,ALASKA,on time,San Diego,212.0
9,ALASKA,delayed,San Diego,20.0


In [4]:
# change the order of row to show by airline and state, ascending 1 means ordered by asce and 0 means ordered by decs
# but using sort_values the index can't change with order
chart = long.sort_values(['Airline','State'], ascending=[1,0])
chart

# reset the index
chart = chart.reset_index(drop=True)
chart

,Airline,State,Destination,Flights
0,ALASKA,on time,Los Angeles,497.0
1,ALASKA,on time,Phoenix,221.0
2,ALASKA,on time,San Diego,212.0
3,ALASKA,on time,San Francisco,503.0
4,ALASKA,on time,Seattle,1841.0
5,ALASKA,delayed,Los Angeles,62.0
6,ALASKA,delayed,Phoenix,12.0
7,ALASKA,delayed,San Diego,20.0
8,ALASKA,delayed,San Francisco,102.0
9,ALASKA,delayed,Seattle,305.0


### Task3: Perform analysis to compare the arrival delays for the two airlines

#### Part1: Which airline had the best overall on time performance?

- groupby() group by Airline and State separately
- sum() used to sum the flight
- **overall performance = sum flights by state / sum flights by Airline** 

In [5]:
# performance by Airline
A_total = chart.groupby(['Airline'])['Flights'].sum().reset_index()
A_total

,Airline,Flights
0,ALASKA,3775.0
1,AMWEST,7225.0


In [6]:
# performance by State
S_total = chart.groupby(['Airline','State'])['Flights'].sum().reset_index()
S_total

,Airline,State,Flights
0,ALASKA,delayed,501.0
1,ALASKA,on time,3274.0
2,AMWEST,delayed,787.0
3,AMWEST,on time,6438.0


In [7]:
# calculate the rate of AL and AM
a = S_total['Flights'][0] / A_total['Flights'][0]
b = S_total['Flights'][1] / A_total['Flights'][0]
c = S_total['Flights'][2] / A_total['Flights'][1]
d = S_total['Flights'][3] / A_total['Flights'][1]
A_Perf = [a,b,c,d]

# print the rate of ontime and delay for Airline and State
S_total = S_total.assign(A_Perf=A_Perf)
S_total[['Airline','State','A_Perf']]

,Airline,State,A_Perf
0,ALASKA,delayed,0.132715
1,ALASKA,on time,0.867285
2,AMWEST,delayed,0.108927
3,AMWEST,on time,0.891073


#### The result shows that Alaska Airlines’ 3775 flights were on-time 86.73% of the time. AMWest’s 7225 flights were on-time 89.11% of the time. 

#### AMWset has the better overall performance

#### Part2: Which airline had the best on time performance?

- group by the destination
- select the ontime data of two airlines
- **performanace = sum flights on time / sum flights by destination**
- show the **difference**

In [8]:
# performance by Destination
D_total = chart.groupby(['Airline','Destination'])['Flights'].sum().reset_index()
D_total

,Airline,Destination,Flights
0,ALASKA,Los Angeles,559.0
1,ALASKA,Phoenix,233.0
2,ALASKA,San Diego,232.0
3,ALASKA,San Francisco,605.0
4,ALASKA,Seattle,2146.0
5,AMWEST,Los Angeles,811.0
6,AMWEST,Phoenix,5255.0
7,AMWEST,San Diego,448.0
8,AMWEST,San Francisco,449.0
9,AMWEST,Seattle,262.0


In [9]:
# select the column of Flights
f = chart['Flights']
D = D_total['Flights']

# calculate the perfomance of ontime by destination
AL_Perf = []
AM_Perf = []
diff = []
for i in range(5):
    # ALASKA ontime rate
    AL_Perf.append(f[i]/D[i])
    # AMWEST ontime rate
    AM_Perf.append(f[i+10]/D[i+5])
    # difference between AL and AM
    diff.append(f[i]/D[i] - f[i+10]/D[i+5])
print(AL_Perf,'\n', AM_Perf)

[0.889087656529517, 0.9484978540772532, 0.9137931034482759, 0.8314049586776859, 0.8578751164958062] 
 [0.8557336621454994, 0.9210275927687916, 0.8549107142857143, 0.7126948775055679, 0.767175572519084]


In [10]:
Pivot = D_total.pivot('Destination','Airline','Flights')
Pivot

Airline,ALASKA,AMWEST
Destination,,
Los Angeles,559.0,811.0
Phoenix,233.0,5255.0
San Diego,232.0,448.0
San Francisco,605.0,449.0
Seattle,2146.0,262.0


In [11]:
# add the performance and difference into the chart, and show result
Pivot = Pivot.assign(AL_Perf = AL_Perf,AM_Perf = AM_Perf, Difference = diff)
Pivot

Airline,ALASKA,AMWEST,AL_Perf,AM_Perf,Difference
Destination,,,,,
Los Angeles,559.0,811.0,0.889088,0.855734,0.033354
Phoenix,233.0,5255.0,0.948498,0.921028,0.027470
San Diego,232.0,448.0,0.913793,0.854911,0.058882
San Francisco,605.0,449.0,0.831405,0.712695,0.118710
Seattle,2146.0,262.0,0.857875,0.767176,0.090700


#### Result:

AMWset has the better overall performance even when looking at individual destinations. But for some destinations the difference in performance is larger than others. Compare Phoenix with the smallest performance difference of 0.0274703 and San Francisco with the largest performance difference of 0.1187101. A cursory review appears to indicate–holding all else equal– that Alaska Airlines performs better when a fewer number of flights are involved and that AM West performs better when larger number of flights are involved. 

### Task4: Consider

I think what i did in the task3 part2 is a good example to use the wide format, the rate of on time is the data people want to know, so we need just a result for them not the whole dataset.